# OCR Correction

In [24]:
import io
from collections import defaultdict
import os, os.path
import sys
sys.path.insert(0, "..")
from string import punctuation
import pprint
import re
from string import punctuation
import collections

from src import iterators

import spacy

import enchant

import fasttext

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk

from gensim import corpora

ImportError: cannot import name 'iterators' from 'src' (/Users/leonardovida/airflow/src/__init__.py)

In [21]:
# Load csv
csv = iterators.iterate_directory("../data/processed/selected_articles/", ".csv")
df = pd.concat([pd.read_csv(c["article_path"]) for c in csv],ignore_index=True)
df.sort_values(by=["count"], ascending=False, inplace=True)
stopword_list = nltk.corpus.stopwords.words('dutch')

NameError: name 'iterators' is not defined

### Pre-process to clean up documents

In [22]:
def remove_punctuation(text):
    """Remove punctuation"""
    no_punct = "".join([c for c in text if c not in punctuation])
    return(no_punct)

In [19]:
df["text_clean"] = df["text"].apply(lambda x: remove_punctuation(x))

In [20]:
tokenizer = RegexpTokenizer(r'[a-zA-Z]+') # No punctuation and no numbers
df["text_clean"] = df["text_clean"].apply(lambda x: tokenizer.tokenize(x.lower()))

In [21]:
def remove_stopwords(text):
    """Remove stopwords as defined by nltk"""
    words = [w for w in text if w not in stopword_list]
    return words

In [22]:
df["text_clean"] = df["text_clean"].apply(lambda x: remove_stopwords(x))

In [23]:
def remove_one_char(text):
    """Eliminates all one char words"""
    words = [w for w in text if len(w)>1]
    return words

In [24]:
df["text_clean"] = df["text_clean"].apply(lambda x: remove_one_char(x))

### Create the corpus

In [39]:
corpus=[]
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    corpus.extend(row["text_clean"])

# Save file
with open('corpus.txt', 'w') as f:
    for item in corpus:
        f.write("%s\n" % item)

In [19]:
dictionary = corpora.Dictionary(corpus)
print(dictionary)

NameError: name 'corpus' is not defined

In [37]:
#vocab = collections.Counter(corpus)
#vocab.most_common(10)

[('zwemvereniging', 1),
 ('denisgcbouw', 1),
 ('fpfp', 1),
 ('jjgjjj', 1),
 ('gffflflgij', 1),
 ('imiihimiiiiiiim', 1),
 ('tegallega', 1),
 ('tertielbenodlgdheden', 1),
 ('verfstoffen', 1),
 ('nieuwaar', 1)]

Inspecting the end of the counter we see tons of problems

In [38]:
list(reversed(vocab.most_common()[-10:]))

[('zwemvereniging', 1),
 ('denisgcbouw', 1),
 ('fpfp', 1),
 ('jjgjjj', 1),
 ('gffflflgij', 1),
 ('imiihimiiiiiiim', 1),
 ('tegallega', 1),
 ('tertielbenodlgdheden', 1),
 ('verfstoffen', 1),
 ('nieuwaar', 1)]

In [16]:
import fasttext
model = fasttext.train_unsupervised('corpus.txt',
                                    epoch=50,
                                    lr=0.01,
                                    wordNgrams=3,
                                   dim=100,
                                   minCount=1,
                                   minn=3,
                                   maxn=6,
                                   ws=6,
                                   neg=20)
model.save_model("fasttext_model.bin")

In [15]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

model = FastText.load_fasttext_format('model_fasttext_2')
 
print(model.wv.most_similar('recharge', topn=5))
print(model.wv.most_similar('reminder', topn=5))
print(model.wv.most_similar('thanks', topn=5))

<ipython-input-15-5115e13380b7>:3: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format('model_fasttext_2')


FileNotFoundError: [Errno 2] No such file or directory: 'model_fasttext_2'

### Load the corpus

In [ ]:
nltk.data.load('corpora/nl_1950/file.txt', format='raw')

words = []
with io.open('corpus.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        words.extend(line.split())
  ​
vocab = collections.Counter(words)
vocab.most_common(10)

## Additional try

In [68]:
def clean_sentences(text):
    tokens = [sent for sent in nltk.sent_tokenize(text)].apply(str)

    sent_list = []
    for sent in tokens:
        sent_str = ''
        for i, word in enumerate(nltk.word_tokenize(sent)):
            # nltk doesn't handle apostrophes correctly
            if word[0] == "'":
                sent_str = sent_str[:-1]

            # only adds words 
            if re.sub('[a-zA-Z]',"", str(word)):
                sent_str += str(word.lower() + ' ')
                sent_list.append(sent_str.strip()).apply(str)

    return str(sent_list)

texts=[[word for word in doc.lower().split() if word not in punctuation] for doc in documents]  #remove punctuation and to lower case
texts=[[word for word in text if word not in stopword_list] for text in texts]  #remove stopwords
texts_try=[re.sub("[^a-zA-Z]", "", text) for text in texts]  # remove special characters and symbols
#texts_try=[[" ".join(word.split()) for word in doc] for doc in documents]   # remove double spaces by splitting the strings into words and joining these words

TypeError: expected string or bytes-like object

In [ ]:
words_nl_clean = [" ".join([w for w in word.split() if len(w)>1]) for word in words_nl_clean] # eliminate all one char words

In [25]:
dictionary = corpora.Dictionary(ldainput_m1)
dictionary.save('/Users/leonardovida/nltk_data/corpora/nl_1950/nl_1950_stop.dict')

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

print(texts)

'k', ' ', 'e', 'i', 'g', 'e', 'n', ' ', 'd', 'r', 'u', 'k', 'k', 'e', 'r', 'u', ' ', 's', 'u', 'c', 'c', 'e', 's', ' ', 'p', 'a', 'r', 't', ' ', 'k', 'r', 'a', 'n', 't', ' ', 'o', 'n', 'z', 'e', ' ', 'g', 'e', 'l', 'u', 'k', ' ', 't', 't', ' ', 'p', 'a', 'r', 't', ' ', 'k', 'r', 'a', 'n', 't', ' ', 'o', 'n', 'z', 'e', ' ', 'g', 'e', 'l', 'u', 'k', ' ', 'a', 'n', ' ', 'm', 'n', 'e', 'r', 'b', 'e', 'l', 'p', 'l', 'e', 'l', 'n', ' ', 'o', 'e', 'n', ' ', 'h', 'a', 'a', 'g', ' ', 'a', 'd', 'u', 'e', 'r', 'e', 'e', 'r', 'e', ' ', 'z', 'i', 'e', 't', ' ', 'v', 'o', 'o', 'r', 'd', 'e', 'e', 'l', ' ', 'p', 'a', 'r', 't', ' ', 'k', 'r', 'a', 'n', 't', ' ', 'o', 'n', 'z', 'e', ' ', 'g', 'e', 'l', 'u', 'k', ' ', 'w', 'e', 'n', 's', 'e', 'n', ' ', 'g', 'r', 'o', 't', 'e', 'r', 'e', ' ', 'g', 'e', 'l', 'u', 'k', 'w', 'e', 'n', 's', 'e', 'n', ' ', 'p', 'a', 'r', 't', 'u', ' ', 'p', 'a', 'r', 't', ' ', 'k', 'r', 'a', 'n', 't', ' ', 'o', 'n', 'z', 'e', ' ', 'g', 'e', 'l', 'u', 'k', ' ', 'w', 'e', 'n', 

In [16]:
ldainput_m2 = [word.split() for word in words_nl_clean]      # words_nl_clean instead of words_nl
id2word_m2 = corpora.Dictionary(ldainput_m2)